In [12]:
%load_ext autoreload
%autoreload 2

# Imports

In [59]:
import pandas as pd
pd.set_option("display.max_columns", None)

import os
from dotenv import load_dotenv
from cryptography.hazmat.primitives import serialization

from src.adapters.brokers.kalshi.base import KalshiHttpClient, Environment
from src.adapters.brokers.kalshi.market_data import *

# Setup Kalshi Client

In [9]:
load_dotenv()
env = Environment.PROD # toggle environment here
KEYID = os.getenv('DEMO_KEYID') if env == Environment.DEMO else os.getenv('PROD_KEYID')
KEYFILE = os.getenv('DEMO_KEYFILE') if env == Environment.DEMO else os.getenv('PROD_KEYFILE')

try:
    with open(KEYFILE, "rb") as key_file:
        private_key = serialization.load_pem_private_key(
            key_file.read(),
            password=None  # Provide the password if your key is encrypted
        )
except FileNotFoundError:
    raise FileNotFoundError(f"Private key file not found at {KEYFILE}")
except Exception as e:
    raise Exception(f"Error loading private key: {str(e)}")

print("PROD_KEYFILE:", os.getenv("PROD_KEYFILE"))
print("PROD_KEYID:", os.getenv("PROD_KEYID"))

PROD_KEYFILE: /Users/luketownsend/Desktop/projects/galton/kalshi-key-2.key
PROD_KEYID: f29e5e7d-1059-4f29-a570-6258379e793a


In [10]:
# Initialize the HTTP client
client = KalshiHttpClient(
    key_id=KEYID,
    private_key=private_key,
    environment=env
)

# Market Data Collection

In [16]:
df = pd.read_parquet(
    "data/aggregated/aggregated_markets_data_processed.parquet"
)

In [190]:
from datetime import datetime, timezone

def iso_to_unix_timestamp(iso_str: str) -> int:
    """
    Convert an ISO 8601 datetime string to a Unix timestamp (seconds since epoch).

    Handles examples like:
      - '2025-11-03T06:24:29Z'
      - '2025-11-03T06:24:29.097644Z'
      - '2025-11-03T22:26:40+00:00'
      - '2025-11-03T22:26:40.64533+00:00'
      - '2025-11-03T17:26:40-05:00'
    """
    s = iso_str.strip()

    # Case 1: UTC with 'Z' suffix
    if s.endswith("Z"):
        for fmt in ("%Y-%m-%dT%H:%M:%S.%fZ", "%Y-%m-%dT%H:%M:%SZ"):
            try:
                dt = datetime.strptime(s, fmt)
                # Z is UTC
                dt = dt.replace(tzinfo=timezone.utc)
                return int(dt.timestamp())
            except ValueError:
                continue

    # Case 2: Offset like +00:00 or -05:00 (no 'Z')
    # Try with fractional seconds first, then without
    for fmt in ("%Y-%m-%dT%H:%M:%S.%f%z", "%Y-%m-%dT%H:%M:%S%z"):
        try:
            dt = datetime.strptime(s, fmt)
            # dt already has tzinfo from %z
            return int(dt.timestamp())
        except ValueError:
            continue

    # If we get here, we really don't recognize the format
    raise ValueError(f"Unrecognized datetime format: {iso_str!r}")



In [112]:
open_time = df[(df["date"] == "2025-09-08") & (df["city"] == "Austin")]["open_time"].values[0]
close_time = df[(df["date"] == "2025-09-08") & (df["city"] == "Austin")]["close_time"].values[0]

series_ticker="KXHIGHAUS"
event_ticker="KXHIGHAUS-25SEP08"

open_time_unix = iso_to_unix_timestamp(open_time)
close_time_unix = iso_to_unix_timestamp(close_time)

In [60]:
market_candlestick_response = get_market_candlestick_data(
    client=client,
    series_ticker="KXHIGHAUS",
    market_ticker="KXHIGHAUS-25SEP08-T93",
    start_ts=open_time_unix,
    end_ts=close_time_unix,
    period_interval=60,
)

In [108]:
import pandas as pd
from typing import List, Dict, Any


def parse_candlesticks(data: List[Dict[str, Any]]) -> pd.DataFrame:
    """
    Flatten Kalshi candlestick response into a pandas DataFrame.
    
    Each nested dict (price, yes_ask, yes_bid) becomes columns with prefixes.
    """
    flat_records = []

    for record in data:
        flat = {}

        # Copy top-level simple fields
        for key, value in record.items():
            if isinstance(value, dict):
                # Flatten nested dict with prefix
                for subkey, subval in value.items():
                    flat[f"{key}_{subkey}"] = subval
            else:
                flat[key] = value

        flat_records.append(flat)

    return pd.DataFrame(flat_records)
df_full_market_candlesticks_v2 = parse_candlesticks(market_candlestick_response['candlesticks'])

In [62]:
df_full_market_candlesticks_v2.head()

,end_period_ts,open_interest,price_close,price_close_dollars,price_high,price_high_dollars,price_low,price_low_dollars,price_max,price_mean,price_mean_dollars,price_min,price_open,price_open_dollars,price_previous,volume,yes_ask_close,yes_ask_close_dollars,yes_ask_high,yes_ask_high_dollars,yes_ask_low,yes_ask_low_dollars,yes_ask_open,yes_ask_open_dollars,yes_bid_close,yes_bid_close_dollars,yes_bid_high,yes_bid_high_dollars,yes_bid_low,yes_bid_low_dollars,yes_bid_open,yes_bid_open_dollars,price_previous_dollars
0,1757257200,2,5.0,0.0500,5.0,0.0500,5.0,0.0500,None,5.0,0.0500,None,5.0,0.0500,NaN,2,7,0.0700,68,0.6800,7,0.0700,68,0.6800,5,0.0500,9,0.0900,0,0.0000,0,0.0000,NaN
1,1757260800,20,8.0,0.0800,8.0,0.0800,7.0,0.0700,None,7.0,0.0767,None,7.0,0.0700,5.0,24,8,0.0800,8,0.0800,7,0.0700,7,0.0700,6,0.0600,6,0.0600,5,0.0500,5,0.0500,0.0500
2,1757264400,38,9.0,0.0900,10.0,0.1000,7.0,0.0700,None,8.0,0.0833,None,8.0,0.0800,8.0,18,9,0.0900,18,0.1800,7,0.0700,8,0.0800,6,0.0600,8,0.0800,5,0.0500,6,0.0600,0.0800
3,1757268000,136,12.0,0.1200,18.0,0.1800,8.0,0.0800,None,13.0,0.1363,None,8.0,0.0800,9.0,104,21,0.2100,21,0.2100,8,0.0800,9,0.0900,8,0.0800,9,0.0900,2,0.0200,6,0.0600,0.0900
4,1757271600,197,17.0,0.1700,17.0,0.1700,11.0,0.1100,None,12.0,0.1251,None,13.0,0.1300,12.0,69,17,0.1700,21,0.2100,13,0.1300,21,0.2100,13,0.1300,13,0.1300,6,0.0600,8,0.0800,0.1200


In [63]:
def parse_dollar_fields(df, name, dollar_fields):
    present_dollar_field_cols = []
    for col in df.columns:
        if col in dollar_fields:
            present_dollar_field_cols.append(col)
        
    df = df[present_dollar_field_cols]

    for col in present_dollar_field_cols:
        renamed_col = f"{name}_{col}"
        df = df.rename(columns={col:renamed_col})

    return df

In [65]:
dollar_fields = ["open_dollars", "close_dollars", "low_dollars", "high_dollars", "mean_dollars", "previous_dollars"]

df_full_market_candlesticks = pd.DataFrame()

for i in range(len(market_candlestick_response["candlesticks"])):
    end_period_ts = market_candlestick_response["candlesticks"][i]["end_period_ts"]
    yes_bid = market_candlestick_response["candlesticks"][i]["yes_bid"]
    yes_ask = market_candlestick_response["candlesticks"][i]["yes_ask"]
    price = market_candlestick_response["candlesticks"][i]["price"]
    volume = market_candlestick_response["candlesticks"][i]["volume"]
    open_interest = market_candlestick_response["candlesticks"][i]["open_interest"]

    df_candlestick_info = pd.DataFrame(
        [{"end_period_ts": end_period_ts, "volume": volume, "open_interest": open_interest}]
    )

    df_yes_bid = parse_dollar_fields(df=pd.DataFrame([yes_bid]), name="yes_bid", dollar_fields=dollar_fields)
    df_yes_ask = parse_dollar_fields(df=pd.DataFrame([yes_ask]), name="yes_ask", dollar_fields=dollar_fields)
    df_price = parse_dollar_fields(df=pd.DataFrame([price]), name="price", dollar_fields=dollar_fields)

    df_candlestick = pd.concat([df_candlestick_info, df_yes_bid, df_yes_ask, df_price], axis=1)

    df_full_market_candlesticks = pd.concat([df_full_market_candlesticks, df_candlestick])

df_full_market_candlesticks["market_ticker"] = market_candlestick_response["ticker"]
df_full_market_candlesticks = df_full_market_candlesticks.reset_index(drop=True)

In [66]:
df_full_market_candlesticks.head()

,end_period_ts,volume,open_interest,yes_bid_close_dollars,yes_bid_high_dollars,yes_bid_low_dollars,yes_bid_open_dollars,yes_ask_close_dollars,yes_ask_high_dollars,yes_ask_low_dollars,yes_ask_open_dollars,price_close_dollars,price_high_dollars,price_low_dollars,price_mean_dollars,price_open_dollars,price_previous_dollars,market_ticker
0,1757257200,2,2,0.0500,0.0900,0.0000,0.0000,0.0700,0.6800,0.0700,0.6800,0.0500,0.0500,0.0500,0.0500,0.0500,NaN,KXHIGHAUS-25SEP08-T93
1,1757260800,24,20,0.0600,0.0600,0.0500,0.0500,0.0800,0.0800,0.0700,0.0700,0.0800,0.0800,0.0700,0.0767,0.0700,0.0500,KXHIGHAUS-25SEP08-T93
2,1757264400,18,38,0.0600,0.0800,0.0500,0.0600,0.0900,0.1800,0.0700,0.0800,0.0900,0.1000,0.0700,0.0833,0.0800,0.0800,KXHIGHAUS-25SEP08-T93
3,1757268000,104,136,0.0800,0.0900,0.0200,0.0600,0.2100,0.2100,0.0800,0.0900,0.1200,0.1800,0.0800,0.1363,0.0800,0.0900,KXHIGHAUS-25SEP08-T93
4,1757271600,69,197,0.1300,0.1300,0.0600,0.0800,0.1700,0.2100,0.1300,0.2100,0.1700,0.1700,0.1100,0.1251,0.1300,0.1200,KXHIGHAUS-25SEP08-T93


In [67]:
df_full_market_candlesticks_v2.head()

,end_period_ts,open_interest,price_close,price_close_dollars,price_high,price_high_dollars,price_low,price_low_dollars,price_max,price_mean,price_mean_dollars,price_min,price_open,price_open_dollars,price_previous,volume,yes_ask_close,yes_ask_close_dollars,yes_ask_high,yes_ask_high_dollars,yes_ask_low,yes_ask_low_dollars,yes_ask_open,yes_ask_open_dollars,yes_bid_close,yes_bid_close_dollars,yes_bid_high,yes_bid_high_dollars,yes_bid_low,yes_bid_low_dollars,yes_bid_open,yes_bid_open_dollars,price_previous_dollars
0,1757257200,2,5.0,0.0500,5.0,0.0500,5.0,0.0500,None,5.0,0.0500,None,5.0,0.0500,NaN,2,7,0.0700,68,0.6800,7,0.0700,68,0.6800,5,0.0500,9,0.0900,0,0.0000,0,0.0000,NaN
1,1757260800,20,8.0,0.0800,8.0,0.0800,7.0,0.0700,None,7.0,0.0767,None,7.0,0.0700,5.0,24,8,0.0800,8,0.0800,7,0.0700,7,0.0700,6,0.0600,6,0.0600,5,0.0500,5,0.0500,0.0500
2,1757264400,38,9.0,0.0900,10.0,0.1000,7.0,0.0700,None,8.0,0.0833,None,8.0,0.0800,8.0,18,9,0.0900,18,0.1800,7,0.0700,8,0.0800,6,0.0600,8,0.0800,5,0.0500,6,0.0600,0.0800
3,1757268000,136,12.0,0.1200,18.0,0.1800,8.0,0.0800,None,13.0,0.1363,None,8.0,0.0800,9.0,104,21,0.2100,21,0.2100,8,0.0800,9,0.0900,8,0.0800,9,0.0900,2,0.0200,6,0.0600,0.0900
4,1757271600,197,17.0,0.1700,17.0,0.1700,11.0,0.1100,None,12.0,0.1251,None,13.0,0.1300,12.0,69,17,0.1700,21,0.2100,13,0.1300,21,0.2100,13,0.1300,13,0.1300,6,0.0600,8,0.0800,0.1200


In [103]:
def parse_event_candlesticks(event_resp: Dict[str, Any]) -> pd.DataFrame:
    """
    Parse an event-level candlestick response that contains multiple markets.

    Expects:
      event_resp["market_tickers"] -> List[str]
      event_resp["candlesticks"]   -> List[List[dict]]  (one list per market)

    Returns a single concatenated DataFrame with a `market_ticker` column.
    """
    market_tickers: List[str] = event_resp["market_tickers"]
    markets_candles: List[List[Dict[str, Any]]] = event_resp["market_candlesticks"]

    dfs: List[pd.DataFrame] = []

    for ticker, candle_list in zip(market_tickers, markets_candles):
        df_market = parse_candlesticks(candle_list)
        df_market["market_ticker"] = ticker
        dfs.append(df_market)

    if not dfs:
        return pd.DataFrame()

    return pd.concat(dfs, ignore_index=True)

In [164]:

def candlestick_data_fetch_loop(
    client: KalshiHttpClient,
    series_ticker: str,
    event_ticker: str,
    start_ts: int,
    end_ts: int,
    period_interval: int = 1,
) -> pd.DataFrame:
    """
    Fetch event-level candlestick data in batches until the full range is covered.

    Returns a DataFrame with all candlestick data for the event.
    """
    df_full_event_candlesticks = pd.DataFrame()
    batch_number = 0
    event_candlestick_response = get_event_candlestick_data(
        client=client,
        series_ticker=series_ticker,
        event_ticker=event_ticker,
        start_ts=start_ts,
        end_ts=end_ts,
        period_interval=period_interval,
    )

    df_event_candlesticks_batch = parse_event_candlesticks(event_candlestick_response)
    df_event_candlesticks_batch['batch_number'] = batch_number

    df_full_event_candlesticks = pd.concat([df_full_event_candlesticks, df_event_candlesticks_batch])

    adjusted_end_ts = event_candlestick_response['adjusted_end_ts']

    while adjusted_end_ts < end_ts:
        batch_number += 1
        event_candlestick_response = get_event_candlestick_data(
            client=client,
            series_ticker=series_ticker,
            event_ticker=event_ticker,
            start_ts=adjusted_end_ts,
            end_ts=end_ts,
            period_interval=period_interval,
        )
        df_event_candlesticks_batch = parse_event_candlesticks(event_candlestick_response)
        df_event_candlesticks_batch['batch_number'] = batch_number

        df_full_event_candlesticks = pd.concat([df_full_event_candlesticks, df_event_candlesticks_batch])

        adjusted_end_ts = event_candlestick_response['adjusted_end_ts']


    df_full_event_candlesticks.drop_duplicates(subset=["market_ticker", "end_period_ts"], inplace=True)
    df_full_event_candlesticks.set_index(["market_ticker", "end_period_ts"], inplace=True)

    filename = f"data/local_data/candlesticks/{event_ticker}_candlesticks.parquet"
    df_full_event_candlesticks.to_parquet(filename)

    print(f"{filename} -- total rows: {len(df_full_event_candlesticks)} -- batches: {batch_number + 1}")


In [165]:
import warnings
warnings.simplefilter("ignore", FutureWarning)

In [166]:
for event in df[df["date"] == "2025-09-12"]["event_ticker"].unique():
    open_time = df[(df["event_ticker"] == event)]["open_time"].values[0]
    close_time = df[df["event_ticker"] == event]["close_time"].values[0]

    open_time_unix = iso_to_unix_timestamp(open_time)
    close_time_unix = iso_to_unix_timestamp(close_time)

    series_ticker = df[df["event_ticker"] == event]["series_id"].values[0]

    candlestick_data_fetch_loop(
        client=client,
        series_ticker=series_ticker,
        event_ticker=event,
        start_ts=open_time_unix,
        end_ts=close_time_unix,
        period_interval=1,
    )

data/local_data/candlesticks/KXHIGHAUS-25SEP12_candlesticks.parquet -- total rows: 7581 -- batches: 3
data/local_data/candlesticks/KXHIGHCHI-25SEP12_candlesticks.parquet -- total rows: 8942 -- batches: 3
data/local_data/candlesticks/KXHIGHDEN-25SEP12_candlesticks.parquet -- total rows: 10122 -- batches: 3
data/local_data/candlesticks/KXHIGHLAX-25SEP12_candlesticks.parquet -- total rows: 8172 -- batches: 3
data/local_data/candlesticks/KXHIGHMIA-25SEP12_candlesticks.parquet -- total rows: 6728 -- batches: 3
data/local_data/candlesticks/KXHIGHNY-25SEP12_candlesticks.parquet -- total rows: 10419 -- batches: 3
data/local_data/candlesticks/KXHIGHPHIL-25SEP12_candlesticks.parquet -- total rows: 5558 -- batches: 3


In [183]:
from pathlib import Path
import glob

folder = Path("data/local_data/candlesticks")

files = glob.glob(str(folder / "*_candlesticks.parquet"))

event_tickers = [
    Path(f).stem.replace("_candlesticks", "")  # remove suffix
    for f in files
]

In [187]:
df[df["date"] == "2025-11-03"]

,market_ticker,event_ticker,status,subtitle,yes_bid,yes_ask,no_bid,no_ask,result,strike_type,floor_strike,open_time,close_time,cap_strike,strike_rank,series_id,city,event_date_str,date,market_type,market_temp,market_floor,market_cap
13350,KXHIGHAUS-25NOV03-T84,KXHIGHAUS-25NOV03,active,85° or above,0,1,99,100,None,greater,84.0,2025-11-03T06:24:29.012384Z,2025-11-04T05:59:00Z,999.0,6,KXHIGHAUS,Austin,25NOV03,2025-11-03,T,84.0,84.0,999.0
13351,KXHIGHAUS-25NOV03-T77,KXHIGHAUS-25NOV03,active,76° or below,0,1,99,100,None,less,-999.0,2025-11-03T06:24:29.012384Z,2025-11-04T05:59:00Z,77.0,1,KXHIGHAUS,Austin,25NOV03,2025-11-03,T,77.0,-999.0,77.0
13352,KXHIGHAUS-25NOV03-B83.5,KXHIGHAUS-25NOV03,active,83° to 84°,0,1,99,100,None,between,83.0,2025-11-03T06:24:29.012384Z,2025-11-04T05:59:00Z,84.0,5,KXHIGHAUS,Austin,25NOV03,2025-11-03,B,83.5,83.0,84.0
13353,KXHIGHAUS-25NOV03-B81.5,KXHIGHAUS-25NOV03,active,81° to 82°,0,1,99,100,None,between,81.0,2025-11-03T06:24:29.012384Z,2025-11-04T05:59:00Z,82.0,4,KXHIGHAUS,Austin,25NOV03,2025-11-03,B,81.5,81.0,82.0
13354,KXHIGHAUS-25NOV03-B79.5,KXHIGHAUS-25NOV03,active,79° to 80°,99,100,0,1,None,between,79.0,2025-11-03T06:24:29.012384Z,2025-11-04T05:59:00Z,80.0,3,KXHIGHAUS,Austin,25NOV03,2025-11-03,B,79.5,79.0,80.0
13355,KXHIGHAUS-25NOV03-B77.5,KXHIGHAUS-25NOV03,active,77° to 78°,0,1,99,100,None,between,77.0,2025-11-03T06:24:29.012384Z,2025-11-04T05:59:00Z,78.0,2,KXHIGHAUS,Austin,25NOV03,2025-11-03,B,77.5,77.0,78.0
13362,KXHIGHCHI-25NOV03-T66,KXHIGHCHI-25NOV03,active,67° or above,0,1,99,100,None,greater,66.0,2025-11-03T06:24:28.926455Z,2025-11-04T05:59:00Z,999.0,6,KXHIGHCHI,Chicago,25NOV03,2025-11-03,T,66.0,66.0,999.0
13363,KXHIGHCHI-25NOV03-T59,KXHIGHCHI-25NOV03,active,58° or below,0,1,99,100,None,less,-999.0,2025-11-03T06:24:28.926455Z,2025-11-04T05:59:00Z,59.0,1,KXHIGHCHI,Chicago,25NOV03,2025-11-03,T,59.0,-999.0,59.0
13364,KXHIGHCHI-25NOV03-B65.5,KXHIGHCHI-25NOV03,active,65° to 66°,0,1,99,100,None,between,65.0,2025-11-03T06:24:28.926455Z,2025-11-04T05:59:00Z,66.0,5,KXHIGHCHI,Chicago,25NOV03,2025-11-03,B,65.5,65.0,66.0
13365,KXHIGHCHI-25NOV03-B63.5,KXHIGHCHI-25NOV03,active,63° to 64°,0,1,99,100,None,between,63.0,2025-11-03T06:24:28.926455Z,2025-11-04T05:59:00Z,64.0,4,KXHIGHCHI,Chicago,25NOV03,2025-11-03,B,63.5,63.0,64.0


In [191]:
from concurrent.futures import ThreadPoolExecutor, as_completed


def run_for_event(event: str, df, client) -> None:
    event_mask = df["event_ticker"] == event
    df_event = df[event_mask]

    open_time = df_event["open_time"].values[0]
    close_time = df_event["close_time"].values[0]

    open_time_unix = iso_to_unix_timestamp(open_time)
    close_time_unix = iso_to_unix_timestamp(close_time)

    series_ticker = df_event["series_id"].values[0]

    candlestick_data_fetch_loop(
        client=client,
        series_ticker=series_ticker,
        event_ticker=event,
        start_ts=open_time_unix,
        end_ts=close_time_unix,
        period_interval=1,
    )

events = df[~df["event_ticker"].isin(event_tickers)]["event_ticker"].unique()

max_workers = 6  # tune based on API rate limits / your machine

with ThreadPoolExecutor(max_workers=max_workers) as executor:
    futures = [
        executor.submit(run_for_event, event, df, client) for event in events
    ]

    # Optional: block until all complete and surface any errors
    for future in as_completed(futures):
        future.result()

data/local_data/candlesticks/KXHIGHAUS-25NOV03_candlesticks.parquet -- total rows: 3527 -- batches: 2
data/local_data/candlesticks/KXHIGHCHI-25NOV03_candlesticks.parquet -- total rows: 3532 -- batches: 2
data/local_data/candlesticks/KXHIGHAUS-25JAN28_candlesticks.parquet -- total rows: 2592 -- batches: 3
data/local_data/candlesticks/KXHIGHAUS-25NOV04_candlesticks.parquet -- total rows: 5390 -- batches: 3
data/local_data/candlesticks/KXHIGHCHI-25NOV04_candlesticks.parquet -- total rows: 5923 -- batches: 3
data/local_data/candlesticks/KXHIGHDEN-25NOV04_candlesticks.parquet -- total rows: 6190 -- batches: 3
data/local_data/candlesticks/KXHIGHDEN-25NOV03_candlesticks.parquet -- total rows: 4523 -- batches: 2
data/local_data/candlesticks/KXHIGHLAX-25NOV03_candlesticks.parquet -- total rows: 6878 -- batches: 2
data/local_data/candlesticks/KXHIGHMIA-25NOV03_candlesticks.parquet -- total rows: 3023 -- batches: 2
data/local_data/candlesticks/KXHIGHLAX-25NOV04_candlesticks.parquet -- total rows:

In [194]:
df["event_ticker"].nunique()

2363

In [192]:
events

array(['KXHIGHAUS-25JAN28', 'KXHIGHAUS-25NOV04', 'KXHIGHAUS-25NOV03',
       'KXHIGHCHI-25NOV04', 'KXHIGHCHI-25NOV03', 'KXHIGHDEN-25NOV04',
       'KXHIGHDEN-25NOV03', 'KXHIGHLAX-25NOV04', 'KXHIGHLAX-25NOV03',
       'KXHIGHMIA-25NOV04', 'KXHIGHMIA-25NOV03', 'KXHIGHNY-25NOV04',
       'KXHIGHNY-25NOV03', 'KXHIGHPHIL-25NOV04', 'KXHIGHPHIL-25NOV03'],
      dtype=object)

In [ ]:
events = df[df["date"].isin(["2025-09-1", "2025-09-13"])]["event_ticker"].unique()

In [180]:
df[(df["date"] >= "2025-09-01") & (df["date"] <= "2025-09-30")]["date"].unique()

array(['2025-09-01', '2025-09-03', '2025-09-02', '2025-09-04',
       '2025-09-06', '2025-09-05', '2025-09-07', '2025-09-08',
       '2025-09-09', '2025-09-10', '2025-09-15', '2025-09-14',
       '2025-09-12', '2025-09-11', '2025-09-20', '2025-09-19',
       '2025-09-18', '2025-09-17', '2025-09-16', '2025-09-30',
       '2025-09-29', '2025-09-28', '2025-09-27'], dtype=object)

In [156]:
open_time

'2025-09-07T14:00:00Z'

In [157]:
series_ticker

'KXHIGHPHIL'

In [ ]:
filename = f"data/local_data/candlesticks/{event_ticker}_candlesticks.parquet"
df_full_event_candlesticks.to_parquet(filename)

In [152]:
df_full_event_candlesticks.to_parquet(filename)

In [120]:
open_time_unix - df_full_event_candlesticks["end_period_ts"].min()

np.int64(-60)

In [121]:
df_full_event_candlesticks["end_period_ts"].max() - close_time_unix

np.int64(0)

In [137]:
df_full_event_candlesticks[df_full_event_candlesticks["batch_number"] == 0]["end_period_ts"].max()

np.int64(1757303580)

In [146]:
df_full_event_candlesticks.pivot_table(index="market_ticker", columns="batch_number", values="end_period_ts", aggfunc="count", margins=True)

batch_number,0,1,2,All
market_ticker,,,,
KXHIGHAUS-25SEP08-B86.5,453,666,667,1786
KXHIGHAUS-25SEP08-B88.5,554,718,672,1944
KXHIGHAUS-25SEP08-B90.5,652,729,312,1693
KXHIGHAUS-25SEP08-B92.5,640,725,304,1669
KXHIGHAUS-25SEP08-T86,401,554,176,1131
KXHIGHAUS-25SEP08-T93,688,662,264,1614
All,3388,4054,2395,9837


In [ ]:
df_full_event_candlesticks.drop_duplicates(subset=["market_ticker", "end_period_ts"], inplace=True)
df_full_event_candlesticks.set_index(["market_ticker", "end_period_ts"], inplace=True)

In [148]:
df_full_event_candlesticks.set_index(["market_ticker", "end_period_ts"], inplace=True)

In [149]:
df_full_event_candlesticks

open_interest  price_close  price_high  \
market_ticker         end_period_ts                                           
KXHIGHAUS-25SEP08-T86 1757253660                 0          NaN         NaN   
                      1757253720                 0          NaN         NaN   
                      1757253960                 0          NaN         NaN   
                      1757254080                 0          NaN         NaN   
                      1757254140                 0          NaN         NaN   
...                                            ...          ...         ...   
KXHIGHAUS-25SEP08-T93 1757375520              2504          NaN         NaN   
                      1757376000              2504          NaN         NaN   
                      1757378100              2504          NaN         NaN   
                      1757381700              2504          NaN         NaN   
                      1757390400              2504          NaN         NaN   

                                     price_low price_max  price_mean  \
market_ticker         end_period_ts                                    
KXHIGHAUS-25SEP08-T86 1757253660           NaN      None         NaN   
                      1757253720           NaN      None         NaN   
                      1757253960           NaN      None         NaN   
                      1757254080           NaN      None         NaN   
                      1757254140           NaN      None         NaN   
...                                        ...       ...         ...   
KXHIGHAUS-25SEP08-T93 1757375520           NaN      None         NaN   
                      1757376000           NaN      None         NaN   
                      1757378100           NaN      None         NaN   
                      1757381700           NaN      None         NaN   
                      1757390400           NaN      None         NaN   

                                    price_min  price_open  price_previous  \
market_ticker         end_period_ts                                         
KXHIGHAUS-25SEP08-T86 1757253660         None         NaN             NaN   
                      1757253720         None         NaN             NaN   
                      1757253960         None         NaN             NaN   
                      1757254080         None         NaN             NaN   
                      1757254140         None         NaN             NaN   
...                                       ...         ...             ...   
KXHIGHAUS-25SEP08-T93 1757375520         None         NaN             4.0   
                      1757376000         None         NaN             4.0   
                      1757378100         None         NaN             4.0   
                      1757381700         None         NaN             4.0   
                      1757390400         None         NaN             4.0   

                                     volume  yes_ask_close  \
market_ticker         end_period_ts                          
KXHIGHAUS-25SEP08-T86 1757253660          0             68   
                      1757253720          0             68   
                      1757253960          0             10   
                      1757254080          0              7   
                      1757254140          0              6   
...                                     ...            ...   
KXHIGHAUS-25SEP08-T93 1757375520          0              1   
                      1757376000          0              1   
                      1757378100          0              1   
                      1757381700          0              1   
                      1757390400          0              1   

                                    yes_ask_close_dollars  yes_ask_high  \
market_ticker         end_period_ts                                       
KXHIGHAUS-25SEP08-T86 1757253660                   0.6800            68   
                      1757253720      

In [130]:
df_full_event_candlesticks[df_full_event_candlesticks["end_period_ts"] == 1757390400]

,end_period_ts,open_interest,price_close,price_high,price_low,price_max,price_mean,price_min,price_open,price_previous,volume,yes_ask_close,yes_ask_close_dollars,yes_ask_high,yes_ask_high_dollars,yes_ask_low,yes_ask_low_dollars,yes_ask_open,yes_ask_open_dollars,yes_bid_close,yes_bid_close_dollars,yes_bid_high,yes_bid_high_dollars,yes_bid_low,yes_bid_low_dollars,yes_bid_open,yes_bid_open_dollars,price_close_dollars,price_high_dollars,price_low_dollars,price_mean_dollars,price_open_dollars,price_previous_dollars,market_ticker,batch_number
784,1757390400,8722,NaN,NaN,NaN,None,NaN,None,NaN,1.0,0,1,0.0100,1,0.0100,1,0.0100,1,0.0100,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,0.0100,KXHIGHAUS-25SEP08-B86.5,2
1457,1757390400,9403,NaN,NaN,NaN,None,NaN,None,NaN,1.0,0,1,0.0100,1,0.0100,1,0.0100,1,0.0100,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,0.0100,KXHIGHAUS-25SEP08-B88.5,2
2398,1757390400,2504,NaN,NaN,NaN,None,NaN,None,NaN,4.0,0,1,0.0100,1,0.0100,1,0.0100,1,0.0100,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,0.0400,KXHIGHAUS-25SEP08-T93,2


In [ ]:
9847 / 60 / 6

27.352777777777778

In [104]:
df_full_event_candlesticks = parse_event_candlesticks(event_candlestick_response)

In [ ]:
adjusted_end_ts = 

In [105]:
(close_time_unix - event_candlestick_response['adjusted_end_ts']) / 60 / 60

0.0

In [106]:
(event_candlestick_response['adjusted_end_ts'] - open_time_unix) / 60 / 60  

38.983333333333334

In [ ]:
event_candlestick_response['adjusted_end_ts']

{'adjusted_end_ts': 1757393940,
 'market_candlesticks': [[{'end_period_ts': 1757257200,
    'open_interest': 0,
    'price': {'close': None,
     'high': None,
     'low': None,
     'max': None,
     'mean': None,
     'min': None,
     'open': None,
     'previous': None},
    'volume': 0,
    'yes_ask': {'close': 3,
     'close_dollars': '0.0300',
     'high': 68,
     'high_dollars': '0.6800',
     'low': 3,
     'low_dollars': '0.0300',
     'open': 68,
     'open_dollars': '0.6800'},
    'yes_bid': {'close': 0,
     'close_dollars': '0.0000',
     'high': 0,
     'high_dollars': '0.0000',
     'low': 0,
     'low_dollars': '0.0000',
     'open': 0,
     'open_dollars': '0.0000'}},
   {'end_period_ts': 1757260800,
    'open_interest': 0,
    'price': {'close': None,
     'high': None,
     'low': None,
     'max': None,
     'mean': None,
     'min': None,
     'open': None,
     'previous': None},
    'volume': 0,
    'yes_ask': {'close': 3,
     'close_dollars': '0.0300',
     '

In [85]:
df_full_event_candlesticks

,end_period_ts,open_interest,price_close,price_high,price_low,price_max,price_mean,price_min,price_open,price_previous,volume,yes_ask_close,yes_ask_close_dollars,yes_ask_high,yes_ask_high_dollars,yes_ask_low,yes_ask_low_dollars,yes_ask_open,yes_ask_open_dollars,yes_bid_close,yes_bid_close_dollars,yes_bid_high,yes_bid_high_dollars,yes_bid_low,yes_bid_low_dollars,yes_bid_open,yes_bid_open_dollars,price_close_dollars,price_high_dollars,price_low_dollars,price_mean_dollars,price_open_dollars,price_previous_dollars,market_ticker
0,1757253660,0,NaN,NaN,NaN,None,NaN,None,NaN,NaN,0,68,0.6800,68,0.6800,68,0.6800,68,0.6800,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,KXHIGHAUS-25SEP08-T86
1,1757253720,0,NaN,NaN,NaN,None,NaN,None,NaN,NaN,0,68,0.6800,68,0.6800,68,0.6800,68,0.6800,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,KXHIGHAUS-25SEP08-T86
2,1757253960,0,NaN,NaN,NaN,None,NaN,None,NaN,NaN,0,10,0.1000,68,0.6800,10,0.1000,68,0.6800,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,KXHIGHAUS-25SEP08-T86
3,1757254080,0,NaN,NaN,NaN,None,NaN,None,NaN,NaN,0,7,0.0700,10,0.1000,7,0.0700,10,0.1000,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,KXHIGHAUS-25SEP08-T86
4,1757254140,0,NaN,NaN,NaN,None,NaN,None,NaN,NaN,0,6,0.0600,10,0.1000,6,0.0600,7,0.0700,0,0.0000,0,0.0000,0,0.0000,0,0.0000,NaN,NaN,NaN,NaN,NaN,NaN,KXHIGHAUS-25SEP08-T86
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3383,1757303280,1058,NaN,NaN,NaN,None,NaN,None,NaN,10.0,0,9,0.0900,9,0.0900,9,0.0900,9,0.0900,7,0.0700,7,0.0700,7,0.0700,7,0.0700,NaN,NaN,NaN,NaN,NaN,0.1000,KXHIGHAUS-25SEP08-T93
3384,1757303400,1058,NaN,NaN,NaN,None,NaN,None,NaN,10.0,0,9,0.0900,9,0.0900,9,0.0900,9,0.0900,7,0.0700,7,0.0700,7,0.0700,7,0.0700,NaN,NaN,NaN,NaN,NaN,0.1000,KXHIGHAUS-25SEP08-T93
3385,1757303460,1058,NaN,NaN,NaN,None,NaN,None,NaN,10.0,0,9,0.0900,9,0.0900,9,0.0900,9,0.0900,7,0.0700,7,0.0700,7,0.0700,7,0.0700,NaN,NaN,NaN,NaN,NaN,0.1000,KXHIGHAUS-25SEP08-T93
3386,1757303520,1058,NaN,NaN,NaN,None,NaN,None,NaN,10.0,0,9,0.0900,9,0.0900,9,0.0900,9,0.0900,7,0.0700,7,0.0700,7,0.0700,7,0.0700,NaN,NaN,NaN,NaN,NaN,0.1000,KXHIGHAUS-25SEP08-T93


In [86]:
3388 / 6

564.6666666666666

In [87]:
564 / 60

9.4